In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [3]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
# from tensorflow.keras.saving import 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
import time
import warnings
warnings.filterwarnings('ignore')

In [5]:
train_img = r"C:\Users\Harikrishnan\OneDrive\Desktop\combined_faces\train"
val_img = r"C:\Users\Harikrishnan\OneDrive\Desktop\combined_faces\test"

# Create an ImageDataGenerator for training (with augmentation)
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,  
    rotation_range=20,  
    width_shift_range=0.2,  
    height_shift_range=0.2,  
    shear_range=0.2,  
    zoom_range=0.2,  
    horizontal_flip=True,  
    fill_mode='nearest'  
)

# ImageDataGenerator for validation (without augmentation)
val_datagen = ImageDataGenerator(rescale=1.0/255.0)

# Train Generator
train_generator = train_datagen.flow_from_directory(
    train_img,
    target_size=(64, 64),  # Resized for most pre-trained models
    batch_size=32,
    class_mode='sparse'
)
# Validation Generator
val_generator = val_datagen.flow_from_directory(
    val_img,
    target_size=(64, 64),
    batch_size=32,
    class_mode='sparse'
)

Found 15983 images belonging to 6 classes.
Found 8255 images belonging to 6 classes.


In [7]:
from tensorflow.keras import models, layers
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = models.Sequential()
model.add(Conv2D(32, (3,3), padding="same", activation='relu', input_shape=(64,64,3)))
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(32, (3,3), activation='relu'))  # Changed kernel size to (3,3) for better learning
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(32, (3,3), activation='relu'))  # Changed kernel size
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(32, (3,3), activation='relu'))  # Changed kernel size
model.add(MaxPooling2D((2,2)))

# Remove one pooling layer to prevent excessive downsampling

model.add(Flatten())  # This should now correctly match the Dense layer

model.add(Dense(512, activation='relu'))     
model.add(Dense(6, activation='softmax'))



In [ ]:
# model = models.Sequential([
#     layers.Conv2D(128, (3,3), activation='relu', input_shape=(224, 224, 3)),
#     layers.MaxPooling2D(2,2),
#     layers.Conv2D(128, (3,3), activation='relu'),
#     layers.MaxPooling2D(2,2),
#     layers.Conv2D(64, (3,3), activation='relu'),
#     layers.MaxPooling2D(2,2),
#     layers.Conv2D(32, (3,3), activation='relu'),
#     layers.MaxPooling2D(2,2),
    
#     layers.Flatten(),  # This should match the expected size
#     layers.Dense(512, activation='relu'),
#     layers.Dense(6, activation='softmax')  # Ensure num_classes matches train_generator.class_indices
# ])

In [9]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [11]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 64, 64, 32)          │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 32, 32, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 30, 30, 32)          │           9,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 15, 15, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 13, 13, 32)          │           9,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 6, 6, 32)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 4, 4, 32)            │           9,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 2, 2, 32)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │          66,048 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 6)                   │           3,078 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 97,766 (381.90 KB)

 Trainable params: 97,766 (381.90 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.fit(train_generator,validation_data=(val_generator),epochs=20)

Epoch 1/20
127/500 ━━━━━━━━━━━━━━━━━━━━ 2:09 346ms/step - accuracy: 0.3291 - loss: 1.6641

In [ ]:
val_acc=model.evaluate(val_generator,verbose=2)
print(val_acc)

In [ ]:
pre=model.predict(val_generator)
pre

In [ ]:
pred=np.argmax(pre,axis=1)
pred

In [ ]:
model.save("age_model.h5")  # Save the model after training

In [ ]:
model = tf.keras.models.load_model("age_model.h5")

In [ ]:
import keras
print(keras.__version__)


In [ ]:
# from sklearn.metrics import accuracy_score,confusion_matrix
# acc=accuracy_score(val_generator,pre)
# acc

In [ ]:
# confusion_matrix(ytest,pred)

In [ ]:
# plt.figure(figsize=(10,10))
# for i in range (100):
#     plt.subplot(10,10,i+1)
#     plt.xticks([])
#     plt.yticks([])
#     plt.imshow(xtest[i],cmap=plt.cm.binary)
#     plt.xlabel(pred[1])

# plt.show()

In [ ]:
# keras.saving.save_model(model, 'my_model.keras')
# model.save('my_model.keras')
# model.save("my_model.h5", save_format="h5")  # Suppress warning for HDF5
# model.save("age_model.h5")  # Save the model after training
# model.save("age_model.keras")

In [ ]:
# model = load_model("age_model.h5")  # Use .h5 only if needed
